In [1]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime

from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_color_value,
    reconstruct_image_from_general,
    plot_full,
    find_nearest_edge,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    find_node_equ,
)
import pandas as pd
from amftrack.pipeline.functions.spore_processing.spore_id import make_spore_data
from amftrack.pipeline.functions.image_processing.hyphae_id_surf import (
    resolve_anastomosis_crossing_by_root,
)
from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Node,
    Edge,
    Hyphae,
    get_distance,
)
from amftrack.util.sys import (
    get_analysis_folders,
    get_time_plate_info_from_analysis,
    get_time_hypha_info_from_analysis,
    get_global_hypha_info_from_analysis,
)

from datetime import datetime
from matplotlib import cm
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from IPython.display import clear_output
from amftrack.pipeline.functions.post_processing.exp_plot import *
import pickle

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2023-07-13 19:32:48.563602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 19:32:49.178971: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/home2/cbisot/miniconda3/envs/amftrack/lib/python3.7/site-packages/cv2/../../lib64:
2023-07-13 19:32:49.179022: I tensorflow/stream_executor/cuda/cudart_stub.cc:29

In [2]:
def get_time(exp, t, tp1):  # redefined here to avoid loop in import
    seconds = (exp.dates[tp1] - exp.dates[t]).total_seconds()
    return seconds / 3600

In [16]:
plates = ["94_20201123"]
directory_targ = directory_project
directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [4]:
all_analysis_folders = get_analysis_folders(directory_targ)
analysis_folders_info = all_analysis_folders.loc[
    all_analysis_folders["unique_id"].isin(plates)
]
folders, time_plate_info = get_time_plate_info_from_analysis(
    analysis_folders_info, use_saved=False
)
folders, time_hypha_info = get_time_hypha_info_from_analysis(
    analysis_folders_info, use_saved=False
)

In [21]:
path_exp = f'{directory_targ}{analysis_folders["path_exp"].iloc[0]}'
exp = pickle.load(open(path_exp, "rb"))
exp.save_location = "/"
try:
    exp.labeled
except AttributeError:
    exp.labeled = True
load_graphs(exp, directory_targ, indexes=range(0, 40))
# load_graphs(exp, directory_targ, indexes=[90, 70, 60])

load_study_zone(exp)

KeyboardInterrupt: 

In [4]:
directory_targ = os.path.join(directory_scratch, "stitch_temp2") + "/"
# directory_targ = directory_project

update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders['unique_id']=='94_20201123']
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]


analysed:   0%|          | 0/341 [00:00<?, ?it/s]

In [7]:
exp = Experiment(directory_targ)
exp.load(folders.iloc[:20])
exp.dates.sort()
# when no width is included
resolve_anastomosis_crossing_by_root(exp, lim_considered=5)

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["date"], format="%d.%m.%Y, %H:%M:"


2020-11-23 12:53:00
2020-11-23 14:00:00
2020-11-24 03:01:00
2020-11-24 05:01:00
2020-11-24 12:02:00
2020-11-24 21:02:00
2020-11-25 18:02:00
2020-11-25 23:02:00
2020-11-26 04:02:00
2020-11-26 06:02:00
2020-11-26 20:09:00
2020-11-27 16:13:00
2020-11-28 01:12:00
2020-11-28 05:13:00
2020-11-29 06:13:00
2020-11-29 18:13:00
2020-12-02 06:09:00
2020-12-03 14:05:00
2020-12-05 21:23:00
2020-12-07 01:24:00


In [ ]:
for t in range(exp.ts):
    exp.load_tile_information(t)
im = exp.get_image(0, 0)
exp.dimX_dimY = im.shape

In [ ]:
def get_length_fast(edge, t):
    return np.linalg.norm(edge.begin.pos(t) - edge.end.pos(t))


fbas = lambda edge: np.log(
    edge.width(edge.ts()[-1])
    * edge.length_um(edge.ts()[-1])
    * edge.end.degree(edge.ts()[-1])
    * edge.begin.degree(edge.ts()[-1])
)  # function to evaluate BASness, typical threshold is 10
f = lambda edge: np.log(
    edge.width(edge.ts()[-1])
    * get_length_fast(edge, edge.ts()[-1])
    * edge.end.degree(edge.ts()[-1])
    * edge.begin.degree(edge.ts()[-1])
)  # function to evaluate BASness, typical threshold is 10

In [8]:
list_hyph = [0, 6, 118, 58, 1, 85]
tf = 90


In [25]:
nx.NetworkXNoPath

networkx.exception.NetworkXNoPath

In [8]:
def get_orientation(hypha, t, start, length=50):
    nodes, edges = hypha.get_nodes_within(t)
    pixel_list_list = []
    #     print(edges[start:])
    for edge in edges[start:]:
        pixel_list_list += edge.pixel_list(t)
    pixel_list = np.array(pixel_list_list)
    vector = pixel_list[min(length, len(pixel_list) - 1)] - pixel_list[0]
    unit_vector = vector / np.linalg.norm(vector)
    vertical_vector = np.array([-1, 0])
    dot_product = np.dot(vertical_vector, unit_vector)
    if (
        vertical_vector[1] * vector[0] - vertical_vector[0] * vector[1] >= 0
    ):  # determinant
        angle = np.arccos(dot_product) / (2 * np.pi) * 360
    else:
        angle = -np.arccos(dot_product) / (2 * np.pi) * 360
    return angle

In [9]:
hyph = [hyph for hyph in exp.hyphaes if hyph.end.label == 0][0]
hyph.root = Node(44, exp)
hyph = [hyph for hyph in exp.hyphaes if hyph.end.label == 6][0]
hyph.root = Node(147, exp)

IndexError: list index out of range

In [10]:
def estimate_angle(exp):
    (
        RH,
        BAS,
        max_speeds,
        total_growths,
        widths,
        lengths,
        branch_frequ,
        select_hyph,
    ) = get_rh_bas(exp)
    branch_root = []
    branch_anastomose = []
    two_time = []
    angles = []
    for rh in RH:
        #     rh = choice(RH)
        t = rh.ts[-1]
        nodes, edges = rh.get_nodes_within(t)
        for i, node in enumerate(nodes[1:-1]):
            found = False
            for hyph in exp.hyphaes:
                if hyph.root.label == node:
                    if found:
                        two_time.append(hyph.root)
                    branch_root.append(hyph.root)
                    if t in hyph.ts:
                        nodes_h, edges_h = hyph.get_nodes_within(t)
                        if len(edges_h) > 0:
                            edge_main = edges[i + 1]
                            edge_branch = edges_h[0]
                            angle_main = get_orientation(rh, t, i + 1, 100)
                            angle_branch = get_orientation(hyph, t, 0, 100)
                            angles.append(((angle_main - angle_branch), (rh, hyph, t)))
                            #                         print(node,edges[i+1],edges_h[0],angle_main-angle_branch)
                            #                         exp.plot([t],[[node,edge_main.begin.label,edge_main.end.label,edge_branch.begin.label,edge_branch.end.label]])
                            found = True
            if not found:
                branch_anastomose.append(Node(node, exp))
    angles_180 = [(angle + 180) % 360 - 180 for angle, infos in angles]
    angles_rh = [(c[0] + 180) % 360 - 180 for c in angles if c[1][1] in RH]
    angles_bas = [(c[0] + 180) % 360 - 180 for c in angles if c[1][1] in BAS]
    return (angles_rh, angles_bas)

In [11]:
from amftrack.pipeline.functions.post_processing.util import measure_length_um

def get_rh_bas(exp):
    select_hyph = get_hyph_infos(exp)
    max_speeds = []
    total_growths = []
    lengths = []
    branch_frequ = []
    hyph_l = []
    RH = []
    BAS = []
    widths = []
    for hyph in exp.hyphaes:
        speeds = [c[2] for c in select_hyph[hyph]]
        ts = [c[0] for c in select_hyph[hyph]]
        tp1s = [c[1] for c in select_hyph[hyph]]
        if len(speeds) > 0:
            node = hyph.end
            length = np.linalg.norm(node.pos(node.ts()[0]) - node.pos(node.ts()[-1]))
            nodes = hyph.get_nodes_within(hyph.ts[-1])[0]
            max_speed = np.max(speeds)
            total_growth = np.sum(
                [
                    speed * get_time(exp, ts[i], tp1s[i])
                    for i, speed in enumerate(speeds)
                ]
            )
            if length>=1500:
                RH.append(hyph)
            else:
                BAS.append(hyph)
            lengths.append(length)
            max_speeds.append(max_speed)
            total_growths.append(total_growth)
            branch_frequ.append((len(nodes) - 1) / (length + 1))
            hyph_l.append(hyph)
            #             widths.append(get_width_hypha(hyph,hyph.ts[-1]))
            widths.append(5)

        else:
            BAS.append(hyph)
    return (
        RH,
        BAS,
        max_speeds,
        total_growths,
        widths,
        lengths,
        branch_frequ,
        select_hyph,
    )
def get_length_um(seg):
    pixel_conversion_factor = 1.725
    pixels = seg
    length_edge = 0
    for i in range(len(pixels) // 10 + 1):
        if i * 10 <= len(pixels) - 1:
            length_edge += np.linalg.norm(
                np.array(pixels[i * 10])
                - np.array(pixels[min((i + 1) * 10, len(pixels) - 1)])
            )
    #         length_edge+=np.linalg.norm(np.array(pixels[len(pixels)//10-1*10-1])-np.array(pixels[-1]))
    return length_edge * pixel_conversion_factor

In [12]:
def get_hyph_infos(exp):
    select_hyph = {}
    for hyph in exp.hyphaes:
        select_hyph[hyph] = []
        for i, t in enumerate(hyph.ts[:-1]):
            tp1 = hyph.ts[i + 1]
            try:
                pixels, nodes = get_pixel_growth_and_new_children(hyph, t, tp1)
                speed = np.sum([get_length_um(seg) for seg in pixels]) / get_time(
                    exp, t, tp1
                )
                select_hyph[hyph].append((t, hyph.ts[i + 1], speed, pixels))
            except nx.NetworkXNoPath:
                pass
    return select_hyph

In [13]:
angles_rh, angles_bas = estimate_angle(exp)

In [14]:
angles_rh

[81.59404923858648,
 66.6042524355534,
 -77.32115012112179,
 -80.69565626974519,
 0.0,
 -81.63529125101888,
 90.35875113716509,
 -83.08052678065816,
 63.2214324339879,
 -66.22401270682784,
 -85.52748709352427,
 -85.52748709352427,
 66.48891034073085,
 -75.8614769941459,
 -66.4572259241238,
 -53.05491108562444,
 67.36554820221608,
 -47.96768199156833,
 -81.62053202841668,
 86.97260657584928,
 0.0,
 -79.15870574121844,
 77.75766880112553,
 65.73004273825777,
 67.72815667539282,
 0.0,
 -74.95343468844287,
 -65.4731358266329,
 -89.18652432618711]

In [18]:
row = [row for index, row in analysis_folders.iterrows()][0]
path = f'{directory_targ}{row["folder_analysis"]}'

In [19]:
path

'/scratch-shared/amftrack/stitch_temp2/Analysis_94_20201123_166_Version1'

In [20]:
np.save(os.path.join(path,"rh_angle"),angles_rh)

In [26]:
from amftrack.pipeline.launching.run_super import run_parallel_post
name_job = "angles"
time = "2:00:00"
list_f = []

list_args = [[]] * len(list_f)
overwrite = True
num_parallel = 30
run_parallel_post(
    "angle_extractor.py",
    list_f,
    list_args,
    [directory_targ, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "global_plate_post_process",
    cpus=128,
    name_job=name_job,
    node="fat",
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 7680 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3062523


In [28]:
angles = []
for rh_label in list_hyph:
    rh = [hyph for hyph in exp.hyphaes if hyph.end.label == rh_label][0]
    #     rh = choice(RH)
    t = tf
    nodes, edges = rh.get_nodes_within(t)
    for i, node in enumerate(nodes[1:-1]):
        found = False
        for hyph in exp.hyphaes:
            if hyph.root.label == node:
                # if found:
                #     two_time.append(hyph.root)
                # branch_root.append(hyph.root)
                if t in hyph.ts:
                    try:
                        nodes_h, edges_h = hyph.get_nodes_within(t)
                        if len(edges_h) > 0:
                            edge_main = edges[i + 1]
                            edge_branch = edges_h[0]
                            angle_main = get_orientation(rh, t, i + 1, 100)
                            angle_branch = get_orientation(hyph, t, 0, 100)
                            angles.append(((angle_main - angle_branch), (rh, hyph, t)))
                            #                         print(node,edges[i+1],edges_h[0],angle_main-angle_branch)
                            #                         exp.plot([t],[[node,edge_main.begin.label,edge_main.end.label,edge_branch.begin.label,edge_branch.end.label]])
                            found = True
                    except nx.NodeNotFound:
                        pass
        # if not found:
        #     branch_anastomose.append(Node(node, exp))
# angles_180 = [(angle + 180) % 360 - 180 for angle, infos in angles]
angles_rh = [(c[0] + 180) % 360 - 180 for c in angles if c[1][1].end.label in list_hyph]
# angles_bas = [(c[0] + 180) % 360 - 180 for c in angles if c[1][1] in BAS]


In [30]:

angles

[(-73.61045966596524, (Hyphae(0,44), Hyphae(1,226), 90)),
 (-68.13619382771023, (Hyphae(0,44), Hyphae(90,134), 90)),
 (47.179261996599834, (Hyphae(0,44), Hyphae(174,237), 90)),
 (96.10314910865742, (Hyphae(118,209), Hyphae(187,207), 90)),
 (255.5538211067594, (Hyphae(118,209), Hyphae(230,191), 90)),
 (88.49818585380926, (Hyphae(118,209), Hyphae(258,194), 90)),
 (-65.56724731167884, (Hyphae(118,209), Hyphae(273,270), 90)),
 (104.41473121842165, (Hyphae(1,226), Hyphae(63,91), 90)),
 (73.19302370567745, (Hyphae(85,60), Hyphae(74,5), 90))]

In [29]:
for end in list_hyph:
    hyph_label = end
    hyph = [hyph for hyph in exp.hyphaes if hyph.end.label == end][0]
    t0 = hyph.ts[0]
    if hyph.end.is_in(t0):
        exp = hyph.experiment
        thresh = 1600
        junctions_found = [hyph.end.neighbours(t0)[0]]
        ts = []
        mult = []
        speeds = []
        print(end)
        select = time_hypha_info.loc[time_hypha_info["end"] == end]

        for t in hyph.ts:
            select_t = select.loc[select["timestep"] == t - 1]
            speed = select_t["speed"].iloc[0] if len(select_t) > 0 else -1
            if t < tf:
                try:
                    G = exp.nx_graph[t]
                    G = G.subgraph(nx.node_connected_component(G, hyph.end.label))
                    tim = hyph.end.ts()[-1]
                    tip = hyph.end
                    if hyph.end.degree(t) == 1 and np.linalg.norm(tip.pos(tim) - tip.pos(t)) >= 40:
                        tot_speeds.append(speed)
                        nodes, edges = hyph.get_nodes_within(t)
                        potentials = []
                        nodes = [Node(node, exp) for node in nodes]
                        try:
                            last_junction_index = nodes.index(junctions_found[-1])
                        except:
                            last_junction_index = 0
                        for node in nodes[last_junction_index + 1 : -1]:
                            dist = np.linalg.norm(node.pos(t) - hyph.end.pos(t))
                            # To avoid detecting two times the same  node with different labels
                            dists_junction_found = [np.inf] + [
                                np.linalg.norm(node.pos(t) - nodo.pos(t))
                                for nodo in junctions_found
                                if nodo.is_in(t)
                            ]
                            if (
                                dist < thresh
                                and min(dists_junction_found) > 40
                                and (node not in junctions_found)
                            ):
                                extra_hypha_neighbours = [
                                    nodo for nodo in node.neighbours(t) if nodo not in nodes
                                ]

                                edges = [
                                    Edge(node, nodo, exp) for nodo in extra_hypha_neighbours
                                ]
                                is_rh = [f(edge) >= 10 for edge in edges]
                                tips = [
                                    nodo
                                    for nodo in extra_hypha_neighbours
                                    if nx.edge_connectivity(G, nodo.label, node.label) == 1
                                ]
                                # if len(tips) == node.degree(t) - 2:

                                if len(tips) == node.degree(t) - 2 and np.any(is_rh):
                                    edge_for_angle = [edge for edge in edges if f(edge) >= 10]
                                    angles = []

[-73.61045966596524]